## NLP Tutorial

NLP - or *Natural Language Processing* - is shorthand for a wide array of techniques designed to help machines learn from text. Natural Language Processing powers everything from chatbots to search engines, and is used in diverse tasks like sentiment analysis and machine translation.

In this tutorial we'll look at this competition's dataset, use a simple technique to process it, build a machine learning model, and submit predictions for a score!

In [101]:
# import numpy as np # linear algebra
import re
import string
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection  #, preprocessing
from spellchecker import SpellChecker

In [102]:
train_df = pd.read_csv("resources/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("resources/kaggle/input/nlp-getting-started/test.csv")

### Cleansing routines

In [103]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
    
def remove_html_tags(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'', text)

def remove_punctuation(text):
    table=str.maketrans('', '', string.punctuation)
    return text.translate(table)
    
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

### prepend keyword value to text

In [104]:
train_df['text'] = (train_df['location'].map(str) + ' ' + train_df['text']) if type(train_df['location']) is str else train_df['text'] 
train_df['text'] = (train_df['keyword'].map(str) + ' ' + train_df['text']) if type(train_df['keyword']) is str else train_df['text']
train_df['text'] = train_df['text'].apply(lambda x : remove_url(x))
train_df['text'] = train_df['text'].apply(lambda x : remove_html_tags(x))
train_df['text'] = train_df['text'].apply(lambda x : remove_emoji(x))
train_df['text'] = train_df['text'].apply(lambda x : remove_punctuation(x))
# train_df['text'] = train_df['text'].apply(lambda x : correct_spellings(x))

### A quick look at our data

Let's look at our data... first, an example of what is NOT a disaster tweet.

In [105]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

And one that is:

In [106]:
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask Canada'

### Building vectors

The theory behind the model we'll build in this notebook is pretty simple: the words contained in each tweet are a good indicator of whether they're about a real disaster or not (this is not entirely correct, but it's a great place to start).

We'll use scikit-learn's `CountVectorizer` to count the words in each tweet and turn them into data our machine learning model can process.

Note: a `vector` is, in this context, a set of numbers that a machine learning model can work with. We'll look at one in just a second.

In [107]:
count_vectorizer = feature_extraction.text.CountVectorizer()

## let's get counts for the first 5 tweets in the data
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [108]:
## we use .todense() here because these vectors are "sparse" (only non-zero elements are kept to save space)
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 53)
[[0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0
  0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


The above tells us that:
1. There are 54 unique words (or "tokens") in the first five tweets.
2. The first tweet contains only some of those unique tokens - all of the non-zero counts above are the tokens that DO exist in the first tweet.

Now let's create vectors for all of our tweets.

In [109]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test_df["text"])

### Our model

As we mentioned above, we think the words contained in each tweet are a good indicator of whether they're about a real disaster or not. The presence of particular word (or set of words) in a tweet might link directly to whether or not that tweet is real.

What we're assuming here is a _linear_ connection. So let's build a linear model and see!

In [110]:
## Our vectors are really big, so we want to push our model's weights
## toward 0 without completely discounting different words - ridge regression 
## is a good way to do this.
clf = linear_model.RidgeClassifier()

Let's test our model and see how well it does on the training data. For this we'll use `cross-validation` - where we train on a portion of the known data, then validate it with the rest. If we do this several times (with different portions) we can get a good idea for how a particular model or method performs.

The metric for this competition is F1, so let's use that here.

In [111]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=10, scoring="f1")
scores

array([0.616179  , 0.48874598, 0.45264848, 0.42424242, 0.51470588,
       0.5       , 0.5221843 , 0.47202797, 0.61742984, 0.66104079])

The above scores aren't terrible! It looks like our assumption will score roughly 0.65 on the leaderboard. There are lots of ways to potentially improve on this (TFIDF, LSA, LSTM / RNNs, the list is long!) - give any of them a shot!

In the meantime, let's do predictions on our training set and build a submission for the competition.

In [112]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [113]:
sample_submission = pd.read_csv("resources/kaggle/input/nlp-getting-started/sample_submission.csv")

In [114]:
sample_submission["target"] = clf.predict(test_vectors)

In [115]:
sample_submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1


In [116]:
sample_submission.to_csv("submission.csv", index=False)

Now, in the viewer, you can submit the above file to the competition! Good luck!